In [3]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
np.set_printoptions(precision=4, suppress=True)
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
file_path ='E:\(세미2)breastcancerdb\data\BREAST1.csv'
file_path

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
C:\Users\user\AppData\Local\Temp\ipykernel_12576\1525194409.py:1: SyntaxWarning: invalid escape sequence '\('
  file_path ='E:\(세미2)breastcancerdb\data\BREAST1.csv'


'E:\\(세미2)breastcancerdb\\data\\BREAST1.csv'

In [9]:
df= pd.read_csv(file_path)
df.head()

,Sex,Age,Race,Marital status,ER,HER2,PR,T/N/M Stage,Behavior code,Sequence number,...,Mets at liver,Mets at lung,Radiation,Surgery Oth Reg/Dis,Surgery Prim Site,Rural-Urban,cancer treatment delay,Income,Survival months,Cancer Site
0,Female,70-74 years,White,Never married,Negative,Negative,Negative,2A,Malignant,2nd of 2 or more primaries,...,No,No,Beam radiation,0,22,Large Metro,28,"$120,000+",59,Breast
1,Female,80-84 years,White,Widowed,Positive,Negative,Positive,0,In situ,4th of 4 or more primaries,...,No,No,Refused (1988+),0,22,Large Metro,27,"$120,000+",52,Breast
2,Female,60-64 years,White,Unknown,Positive,Negative,Positive,1A,Malignant,2nd of 2 or more primaries,...,No,No,None/Unknown,0,0,Large Metro,Unable to calculate,"$120,000+",38,Breast
3,Female,70-74 years,Asian or Pacific Islander,Widowed,Positive,Negative,Positive,1A,Malignant,3rd of 3 or more primaries,...,No,No,None/Unknown,0,41,Large Metro,27,"$120,000+",49,Breast
4,Female,45-49 years,Asian or Pacific Islander,Married (including common law),Positive,Negative,Positive,1A,Malignant,2nd of 2 or more primaries,...,No,No,None/Unknown,0,49,Large Metro,9,"$120,000+",13,Breast
